In [10]:
import fitz
import pandas as pd
import re

In [11]:
valid_college_codes = pd.read_csv("data/college_codes.csv", header=None).squeeze().tolist()

In [12]:
def extract_cleaned_table_text(pdf_path, max_pages=None):
    doc = fitz.open(pdf_path)
    table_text = ""
    
    header_keywords = [
        "TAMILNADU ENGINEERING ADMISSIONS",
        "DIRECTORATE OF TECHNICAL EDUCATION",
        "ACADEMIC SEAT MATRIX AFTER"
    ]

    for page_num in range(min(max_pages or len(doc), len(doc))):
        text = doc[page_num].get_text()
        lines = text.splitlines()
        
        cleaned_lines = [
            line for line in lines
            if not any(keyword in line.upper() for keyword in header_keywords)
            and not re.match(r"Page\s+\d+\s+of\s+\d+", line, re.IGNORECASE)
        ]
        
        table_text += "\n".join(cleaned_lines) + "\n"
    
    return table_text


In [13]:
import re
import pandas as pd

# valid_college_codes = set([1, 2, 3, 4, 5, 1399, ...])  # populate externally

# Detect suspect branch names with trailing digits
def is_suspect_branch_name(name):
    clean = name.strip().lower()
    return bool(re.search(r"(\d+$)", clean))

# Parser
def parse_multiline_seat_data(text):
    lines = [ln.strip() for ln in text.splitlines() if ln.strip()]
    records, malformed_logs, branch_digit_logs = [], [], []
    i = 0
    while i < len(lines) - 11:
        start_i = i
        # skip junk lines
        if re.match(r'^(\d+\s+){3,}', lines[i]): i += 1; continue
        tokens = lines[i].split()
        if not (len(tokens)==1 and tokens[0].isdigit() and int(tokens[0]) in valid_college_codes): i+=1; continue
        college_code = tokens[0]; i+=1
        # college name
        cn=[]
        while i<len(lines) and not re.fullmatch(r"[A-Z]{2}", lines[i]): cn.append(lines[i]); i+=1
        college_name = " ".join(cn)
        if i>=len(lines): break
        branch_code = lines[i]; i+=1
        # branch name + numeric block
        bn_tokens, window, numeric_block = [], [], []
        while i<len(lines):
            tok = lines[i]
            # standalone number
            if re.fullmatch(r"\d+", tok):
                window.append(int(tok))
                if len(window)==8: numeric_block=window; i+=1; break
                i+=1; continue
            # token with trailing number
            m = re.match(r"^(.+?)\s+(\d+)$", tok)
            if m and not window:
                bn_tokens.append(m.group(1))
                window.append(int(m.group(2)))
                i+=1; continue
            if window: break
            bn_tokens.append(tok)
            i+=1
        full_branch = " ".join(bn_tokens)
        # remove any trailing numbers
        full_branch = re.sub(r"\s*\d+(?:\s+\d+)*$", "", full_branch)
        # log if still ends with number
        if is_suspect_branch_name(full_branch):
            branch_digit_logs.append({'college_code':college_code,'branch_code':branch_code,'branch_name':full_branch})
        # validate numeric_block
        if len(numeric_block)!=8:
            nums = [int(n) for n in re.findall(r"\b\d+\b", " ".join(lines[start_i:i+1]))]
            if len(nums)>=8: numeric_block=nums[-8:]
            else:
                malformed_logs.append({'start_index':start_i,'college_code':college_code,'branch_code':branch_code,'branch_name':full_branch,'raw_lines':lines[start_i:i+1]})
                continue
        rec = {"college_code":college_code,"college_name":college_name,
               "branch_code":branch_code,"branch_name":full_branch,
               "OC":numeric_block[0],"BC":numeric_block[1],"BCM":numeric_block[2],
               "MBC":numeric_block[3],"SC":numeric_block[4],"SCA":numeric_block[5],
               "ST":numeric_block[6],"Total":numeric_block[7]}
        records.append(rec)
    df = pd.DataFrame(records)
    dupes = df[df.duplicated(['college_code','branch_code'],keep=False)]
    df['ComputedTotal']=df[['OC','BC','BCM','MBC','SC','SCA','ST']].sum(axis=1)
    mismatches = df[df['ComputedTotal']!=df['Total']]
    return df.drop(columns='ComputedTotal'), malformed_logs, dupes, mismatches, branch_digit_logs


In [14]:
# === Run ===
pdf_path = "data/GOVT_ACADEMIC_SEAT_MATRIX_2025-07-16.pdf"
raw_text = extract_cleaned_table_text(pdf_path, max_pages=300)
df_govt, bad_govt, dupes_govt, mismatches_govt, branch_digit = parse_multiline_seat_data(raw_text)


# === Save + View ===
df_govt.to_csv("data/df_govt.csv", index=False)
df_govt.head(5)

,college_code,college_name,branch_code,branch_name,OC,BC,BCM,MBC,SC,SCA,ST,Total
0,1,University Departments of Anna University Che...,BY,BIO MEDICAL ENGINEERING (SS),2,0,0,1,1,0,0,4
1,1,University Departments of Anna University Che...,CE,CIVIL ENGINEERING,2,1,0,1,1,0,0,5
2,1,University Departments of Anna University Che...,CM,COMPUTER SCIENCE AND ENGINEERING (SS),3,3,1,2,2,0,0,11
3,1,University Departments of Anna University Che...,CS,COMPUTER SCIENCE AND ENGINEERING,2,1,0,1,0,0,0,4
4,1,University Departments of Anna University Che...,EC,ELECTRONICS AND COMMUNICATION ENGINEERING,1,1,0,1,1,0,1,5


In [15]:
pdf_path_total = "data/GENERAL_ACADEMIC_SEAT_MATRIX_2025-07-16.pdf"
raw_text_total = extract_cleaned_table_text(pdf_path_total, max_pages=306)
df_total, bad_total, dupes_total, mismatches_total, branch_digit_total = parse_multiline_seat_data(raw_text_total)

# === Save + View ===
df_total.to_csv("data/df_total.csv", index=False)
df_total.head(5)

,college_code,college_name,branch_code,branch_name,OC,BC,BCM,MBC,SC,SCA,ST,Total
0,1,University Departments of Anna University Che...,BY,BIO MEDICAL ENGINEERING (SS),17,17,3,12,9,2,1,61
1,1,University Departments of Anna University Che...,CE,CIVIL ENGINEERING,18,18,2,12,9,2,1,62
2,1,University Departments of Anna University Che...,CM,COMPUTER SCIENCE AND ENGINEERING (SS),36,31,4,23,19,4,1,118
3,1,University Departments of Anna University Che...,CS,COMPUTER SCIENCE AND ENGINEERING,17,16,2,12,9,2,1,59
4,1,University Departments of Anna University Che...,EC,ELECTRONICS AND COMMUNICATION ENGINEERING,18,16,2,13,8,2,0,59


In [16]:
import pandas as pd

def export_diagnostics_to_excel(df, malformed_logs, duplicate_keys, mismatch_rows, branch_digit_logs, filename="seat_matrix_diagnostics.xlsx"):
    # Format malformed logs
    malformed_df = pd.DataFrame([
        {
            "start_index": row.get("start_index"),
            "college_code": row.get("college_code"),
            "branch_code": row.get("branch_code"),
            "branch_name": row.get("branch_name"),
            "college_name": row.get("college_name"),
            "raw_text": " | ".join(row.get("raw_lines", []))
        }
        for row in malformed_logs
    ])

    # Digit contamination logs
    digit_issue_df = pd.DataFrame(branch_digit_logs)

    # Write all to Excel
    with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
        df.to_excel(writer, sheet_name='Cleaned_Data', index=False)
        malformed_df.to_excel(writer, sheet_name='Malformed_Records', index=False)
        duplicate_keys.to_excel(writer, sheet_name='Duplicates', index=False)
        mismatch_rows.to_excel(writer, sheet_name='Mismatch_Totals', index=False)
        digit_issue_df.to_excel(writer, sheet_name='Branch_Digit_Issues', index=False)

    print(f"✅ Exported all diagnostics to {filename}")


In [17]:
export_diagnostics_to_excel(df_total, bad_total, dupes_total, mismatches_total, branch_digit_total, filename="data/total_seat_matrix_diagnostics.xlsx")

✅ Exported all diagnostics to data/total_seat_matrix_diagnostics.xlsx


In [18]:
export_diagnostics_to_excel(df_govt, bad_govt, dupes_govt, mismatches_govt, branch_digit, filename="data/govt_seat_matrix_diagnostics.xlsx")

✅ Exported all diagnostics to data/govt_seat_matrix_diagnostics.xlsx
